In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
import pandas as pd
from skimage import data, util
from skimage.measure import label, regionprops, regionprops_table
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets


def calc_iou(img1, img2):
    img1 = img1 > 0
    img2 = img2 > 0
    img1 = img1.astype(np.uint8)
    img2 = img2.astype(np.uint8)
    intersection = cv2.bitwise_and(img1, img2)
    union = cv2.bitwise_or(img1, img2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def fourier_descriptor(x, y):
    complex_ = np.array([complex(i, j) for i, j in zip(x, y)])
    F = np.fft.fft(complex_)
    F_positive = abs(F)
    scale = abs(F[1])
    F_trunc = F_positive[1:7]
    F_final = F_trunc / scale
    return F_final

def get_key_from_value(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None


# Phase 1 

iou_val = []
# Load images
dirs = [f for f in os.listdir('.') if f == 'output_pre']

for dir in dirs:
    os.chdir(dir)
    idirs = [f for f in os.listdir('../' + dir)]    
    for idir in idirs:
        os.chdir(idir)
        imgs = [f for f in os.listdir('../' + idir) if f.endswith('.bmp')]
        path = "../../pics/" + idir
#         os.mkdir(path)
        for img in imgs:
            
            # Store name of image
            name = img
            
            # Read image
            image = cv2.imread(img, cv2.IMREAD_COLOR)
            gnd_tru = cv2.imread("../../output/" + idir + "/" + name, cv2.IMREAD_COLOR)

            
            # Convert the image to the HSV color space
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
            # Define the lower and upper bounds of the purple color in the HSV color space
            lower_bound = np.array([129,40,0])
            upper_bound = np.array([174, 255, 255])
        
            # Create the mask
            mask = cv2.inRange(hsv, lower_bound, upper_bound)
        
            # Extract the purple pixels using the mask
            purple = cv2.bitwise_and(image, image, mask=mask)
         # Remove noise
            purple = cv2.GaussianBlur(purple,(5,5),5)
        
            # Convert to grayscale and apply thresholding
            gray = cv2.cvtColor(purple, cv2.COLOR_BGR2GRAY)
        
            # Threshold the image to create a topographic map
            ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
            #erosion
            kernel_size =[5,5]
            ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
            eroded = cv2.erode(thresh, ellipse_kernel, iterations = 2)
            #dilation
            kernel_size =[5,5]
            ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size) 
            dilated = cv2.dilate(eroded, ellipse_kernel, iterations = 2)
        
            rgb = cv2.cvtColor(dilated, cv2.COLOR_BGR2RGB)
        
            # Save images to dedicated folder
#             cv2.imwrite(os.path.join(path,name), rgb)
        
            #iou
            iou = calc_iou(gnd_tru, rgb)
            iou_val.append(iou)
        os.chdir('..')
    os.chdir('..')
                 
avg = np.mean(iou_val) * 100
print(avg)
# Phase 2

os.chdir("./pics")

features_table = []
table_indices = []
targets = {0:"baso", 1:"eosi", 2:"lymp", 3:"mixt", 4:"mono", 5:"neut"}
target_codes = []


idirs = [f for f in os.listdir('.')]
for idir in idirs:
    os.chdir(idir)
    imgs = [f for f in os.listdir('../' + idir) if f.endswith('.bmp')]
    for img in imgs:
#         path = ".\pics\\" + idir + "\\" + img
        img_full_name =img + "-" + idir
        name = img
        img = cv2.imread(name, cv2.IMREAD_COLOR)
        img_grayed = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        
        #Exctract features
        test_label = label(img_grayed)
        test_regions = regionprops(test_label)
        
        if test_regions != []:
            #  Find countours on GRAY_SCALE BINARY image
            contours, hierarchy = cv2.findContours(image=img_grayed, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
            # Draw the contours on COLORED ORIGINAL image
            cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=1, lineType=cv2.LINE_AA)

            # Label the GRAY_SCALE BINARY image
            label_img = label(img_grayed)
            regions = regionprops(label_img)
            
            
            temp_x = contours[0][:, 0][:, 0]
            temp_y = contours[0][:, 0][:, 1]
            f_descriptors = fourier_descriptor(temp_x, temp_y) 
            
            # regions > 1
            if len(regions) > 1:
                    target_codes.append(get_key_from_value(targets, idir))
                    
            # regions == 1
            else:
                # feature table
                props_dict = regionprops_table(label_img, img,
                                               properties=['area', 'area_convex', 'solidity', 'perimeter', 'eccentricity'])
                # Add the scikit dict built-in properties to the current image features
                image_features = []
                for prop, values in props_dict.items():
                    for i, value in enumerate(values):
                        image_features.append(value)
                # Append the extra properties as fourier to the current image features
                for f in f_descriptors:
                    image_features.append(f)
                            
                target_codes.append(get_key_from_value(targets, idir))
                # no regions --> fill with 0s
        else:
            image_features = []
            image_features = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
            
            # image_features.append(current_directory)
            target_codes.append(get_key_from_value(targets, idir))

        # : Do anyway  
        # Append the image name to the indices for dataframe printing later
        table_indices.append(img_full_name)
        # Append each row of features (1 image features) to all the feature table
        features_table.append(image_features)
    
    
    os.chdir('..')
os.chdir('..')

df = pd.DataFrame(features_table)
df.columns =  ['Area', 'Area_convex', 'Solidity', 'Perimeter', 'Eccentricity', 'F1',  'F2', 'F3', 'F4','F5','F6']
df.index = table_indices
print(df)


X = np.array(features_table)
y = np.array(target_codes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = svm.SVC(kernel="linear", C = 1)
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
accuracy = clf.score(X_test, y_test) * 100
print("Accuracy: ",accuracy)


                

59.65184803522964
               Area  Area_convex  Solidity   Perimeter  Eccentricity   F1  \
1.bmp-baso   5623.0       5736.0  0.980300  284.693434      0.591429  1.0   
10.bmp-baso  3770.0       3816.0  0.987945  227.480231      0.478593  1.0   
11.bmp-baso  3744.0       3790.0  0.987863  224.994949      0.394318  1.0   
12.bmp-baso  3031.0       3070.0  0.987296  202.166522      0.366782  1.0   
13.bmp-baso  4813.0       4876.0  0.987080  258.651804      0.568704  1.0   
...             ...          ...       ...         ...           ...  ...   
50.bmp-neut  3826.0       3938.0  0.971559  232.409163      0.477564  1.0   
6.bmp-neut   3573.0       3624.0  0.985927  221.237590      0.410531  1.0   
7.bmp-neut   3220.0       3290.0  0.978723  212.509668      0.649061  1.0   
8.bmp-neut      0.0          0.0  0.000000    0.000000      0.000000  0.0   
9.bmp-neut   3669.0       3721.0  0.986025  226.308658      0.577898  1.0   

                   F2        F3        F4        F5      